<center><h2> Matrix transfert method step by step based on Spectroscopic Ellipsometry (H. Fujiwara)</h2></center>

Here is a step by step demonstration of the matrix transfert method using Berreman's formalism for an anisotropic layer on an isotropic substrate to calculate the ellipsometric matrices psi and delta. The example is based on the Matrix transfert method as illustrated in section 6.4.2 of the book by H.Fujiwara, Spectroscopic Ellipsometry (John Wiley & Sons, Ltd, 2007). More information can be found on this method in section 6.3

In [46]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from scipy.optimize import leastsq
import math
import matplotlib.pyplot as plt
%matplotlib inline

#----Various constant definitions----#
c=299792458 #speed of light
lam=619.92e-9#definition of the wavelength array in nm
omega=2*c*math.pi/lam #definition of the angular frequency array
theta_i=math.radians(70)#Incidence angle
d=1000e-10#Thickness of the layer in m

#Definition of the complex refractive index of the substrate
nsub=3.898
n_i=1
ksub=0.016
Nsub=nsub+ksub*1j

#Definition of the dielectric matrix of the anisotropic layer
nx=ny=2.0 
nz=2.5
kx=ky=kz=0
epsx=nx**2-kx**2+1j*(2*nx*kx)
epsy=ny**2-ky**2+1j*(2*ny*ky)
epsz=nz**2-kz**2+1j*(2*nz*kz)
epsp =np.matrix([[epsx,0,0],[0, epsy,0],[0, 0,epsz]])

#----Definition of the Euler angles----#
phi_E=math.radians(45)
theta_E=math.radians(45)
psi_E=math.radians(0)

#Definition of coefficients for the Euler matrix calculation
c1=np.cos(phi_E)
c2=np.cos(theta_E)
c3=np.cos(psi_E)
s1=np.sin(phi_E)
s2=np.sin(theta_E)
s3=np.sin(psi_E)
Euler=np.matrix([[c1*c3-(c2*s1*s3),-c1*s3-(c2*c3*s1),s1*s2],
        [c3*s1+(c1*c2*s3),c1*c2*c3-(s1*s3),-c1*s2],
        [s2*s3, c3*s2,c2]])

#Computation of the dielectric tensor given the euler angle of the thin film
eps=np.dot(np.dot(Euler,epsp),Euler.transpose())

eps

matrix([[ 4.5625    +0.j, -0.5625    +0.j,  0.79549513+0.j],
        [-0.5625    +0.j,  4.5625    +0.j, -0.79549513+0.j],
        [ 0.79549513+0.j, -0.79549513+0.j,  5.125     +0.j]])

In [45]:
#Computation of the partial transfert matrix
def Ti():
    Kxx=n_i*np.sin(theta_i) 
    #Calculus of the delta matrix#
    delta=np.matrix([[-Kxx*(eps[2,0]/eps[2,2]),-Kxx*(eps[2,1]/eps[2,2]),0,1-(Kxx*Kxx/eps[2,2])],
                  [0,0,-1,0],
                  [(eps[1,2]*eps[2,0]/eps[2,2])-eps[1,0],Kxx*Kxx-eps[1,1]+(eps[1,2]*eps[2,1]/eps[2,2]),0,(Kxx*eps[1,2]/eps[2,2])],
                  [(-eps[0,2]*eps[2,0]/eps[2,2])+eps[0,0],-(eps[0,2]*eps[2,1]/eps[2,2])+eps[0,1],0,(-Kxx*eps[0,2]/eps[2,2])]]) 
    #Calculus of the eigenvalues matrix#
    q=np.linalg.eig(delta)[0] 
    
    beta_0=-q[1]*q[2]*q[3]*np.exp(-d*1j*q[0]*omega/c)/((q[0]-q[1])*(q[0]-q[2])*(q[0]-q[3]))-q[2]*q[3]*q[0]*np.exp(-d*1j*q[1]*omega/c)/((q[1]-q[2])*(q[1]-q[3])*(q[1]-q[0]))-q[3]*q[0]*q[1]*np.exp(-d*1j*q[2]*omega/c)/((q[2]-q[3])*(q[2]-q[0])*(q[2]-q[1]))-q[0]*q[1]*q[2]*np.exp(-d*1j*q[3]*omega/c)/((q[3]-q[0])*(q[3]-q[1])*(q[3]-q[2])) 

    beta_1=((q[1]*q[2])+(q[1]*q[3])+(q[2]*q[3]))*np.exp(-d*1j*q[0]*omega/c)/((q[0]-q[1])*(q[0]-q[2])*(q[0]-q[3]))+((q[2]*q[3])+(q[2]*q[0])+(q[3]*q[0]))*np.exp(-d*1j*q[1]*omega/c)/((q[1]-q[2])*(q[1]-q[3])*(q[1]-q[0]))+((q[3]*q[0])+(q[3]*q[1])+(q[0]*q[1]))*np.exp(-d*1j*q[2]*omega/c)/((q[2]-q[3])*(q[2]-q[0])*(q[2]-q[1]))+((q[0]*q[1])+(q[0]*q[2])+(q[1]*q[2]))*np.exp(-d*1j*q[3]*omega/c)/((q[3]-q[0])*(q[3]-q[1])*(q[3]-q[2])) 

    beta_2=-(q[1]+q[2]+q[3])*np.exp(-d*1j*q[0]*omega/c)/((q[0]-q[1])*(q[0]-q[2])*(q[0]-q[3]))-(q[2]+q[3]+q[0])*np.exp(-d*1j*q[1]*omega/c)/((q[1]-q[2])*(q[1]-q[3])*(q[1]-q[0]))-(q[3]+q[0]+q[1])*np.exp(-d*1j*q[2]*omega/c)/((q[2]-q[3])*(q[2]-q[0])*(q[2]-q[1]))-(q[0]+q[1]+q[2])*np.exp(-d*1j*q[3]*omega/c)/((q[3]-q[0])*(q[3]-q[1])*(q[3]-q[2])) 

    beta_3=np.exp(-d*1j*q[0]*omega/c)/((q[0]-q[1])*(q[0]-q[2])*(q[0]-q[3]))+np.exp(-d*1j*q[1]*omega/c)/((q[1]-q[2])*(q[1]-q[3])*(q[1]-q[0]))+np.exp(-d*1j*q[2]*omega/c)/((q[2]-q[3])*(q[2]-q[0])*(q[2]-q[1]))+np.exp(-d*1j*q[3]*omega/c)/((q[3]-q[0])*(q[3]-q[1])*(q[3]-q[2]))
    Ti=beta_0*np.identity(4)+beta_1*delta+beta_2*np.dot(delta,delta)+beta_3*np.dot(delta,np.dot(delta,delta)) 
    return Ti
Ti()

matrix([[-0.35178828-5.71947547e-02j,  0.09057131-9.38065178e-04j,
          0.03336196+4.37442098e-02j,  0.05608777-3.97092737e-01j],
        [ 0.10420181+7.95632408e-02j, -0.32602991-1.14480661e-02j,
          0.00372234+5.01675329e-01j, -0.03336196-4.37442098e-02j],
        [ 0.16165901-1.44611770e-02j, -0.01660869+1.75314887e+00j,
         -0.32602991-1.14480661e-02j, -0.09057131+9.38065178e-04j],
        [ 0.3001355 -2.12045205e+00j, -0.16165901+1.44611770e-02j,
         -0.10420181-7.95632408e-02j, -0.35178828-5.71947547e-02j]])

In [44]:
#Computation of the partial transfert matrix
def Tp():
    nTi=Ti()
    nt=Nsub
    costheta_t=np.sqrt(1-(n_i*np.sin(theta_i)**2/(nt**2))) 
    Lt=np.matrix([[0,0,costheta_t,-costheta_t],[1,1,0,0],[-nt*costheta_t,nt*costheta_t,0,0],[0,0,nt,nt]]) 
    Li=1/2*np.matrix([[0,1,-1/(n_i*np.cos(theta_i)),0],[0,1,1/(n_i*np.cos(theta_i)),0],[1/np.cos(theta_i),0,0,n_i],[-1/np.cos(theta_i),0,0,n_i]]) 
    Tp=np.dot(Li,np.dot(nTi,Lt)) 
    return Tp
Tp()

matrix([[-1.94445172-3.58878653j,  1.66698237-1.54852591j,
          0.27265891-0.02967091j,  0.63028026-0.14783193j],
        [ 1.61261075+1.67941887j, -1.98720122+3.43499744j,
         -0.30089468-0.06413535j, -0.86073454-0.10045926j],
        [ 0.06456899-0.08551493j,  0.03858481+0.09723339j,
         -0.70946401-3.48600176j, -0.00308352-1.26560995j],
        [ 0.16665999+0.40268499j, -0.5931318 -0.38548109j,
         -0.36908616+1.19958362j, -1.6590775 +3.09488056j]])

In [54]:
def Jr():
    T=Tp()
    #Calculus of the Jones reflectance coefficients at the interface#
    Rpp=(T[0,0]*T[3,2]-T[0,2]*T[3,0])/(T[0,0]*T[2,2]-T[0,2]*T[2,0]) 
    Rsp=(T[0,0]*T[1,2]-T[0,2]*T[1,0])/(T[0,0]*T[2,2]-T[0,2]*T[2,0]) 
    Rss=(T[1,0]*T[2,2]-T[1,2]*T[2,0])/(T[0,0]*T[2,2]-T[0,2]*T[2,0]) 
    Rps=(T[2,2]*T[3,0]-T[2,0]*T[3,2])/(T[0,0]*T[2,2]-T[0,2]*T[2,0]) 
    Jr=[[Rpp,Rps],[Rsp,Rss]]
    return Jr

def Jt():
    T=Tp()
    #Calculus of the Jones tansmittance coefficients at the interface#
    Tss=T[2,2]/(T[0,0]*T[2,2]-T[0,2]*T[2,0]) 
    Tsp=-T[2,0]/(T[0,0]*T[2,2]-T[0,2]*T[2,0]) 
    Tps=-T[0,2]/(T[0,0]*T[2,2]-T[0,2]*T[2,0]) 
    Tpp=T[0,0]/(T[0,0]*T[2,2]-T[0,2]*T[2,0]) 
    Jt=[[Tpp,Tps],[Tsp,Tss]]
    return Jt

Jr()

[[(-0.3106439496927664-0.1607400276220289j),
  (-0.10665214015582612-0.0019272000962895618j)],
 [(0.04218588638242871-0.03589322282767148j),
  (-0.5512905338559693+0.15074689671427677j)]]

In [50]:
def psi():
    J=Jr()
    psi_pp=np.arctan2(np.absolute(J[0][0]), np.absolute(J[1][1]))* 180 / np.pi
    psi_ps=np.arctan2(np.absolute(J[0][1]), np.absolute(J[1][1]))* 180 / np.pi
    psi_sp=np.arctan2(np.absolute(J[1][0]), np.absolute(J[1][1]))* 180 / np.pi
    return {'psi_pp':psi_pp,'psi_ps':psi_ps,'psi_sp':psi_sp}

def delta():
    J=Jr()
    rp= J[0][0]
    rs= J[1][1] 
    delta_pp=-np.angle(J[0][0]/J[1][1],deg=True)
    delta_ps=-np.angle(J[0][1]/J[1][1],deg=True)
    delta_sp=-np.angle(J[1][0]/J[1][1],deg=True)
    return {'delta_pp':delta_pp,'delta_ps':delta_ps,'delta_sp':delta_sp}

In [51]:
psi()

{'psi_pp': 31.46598227397841,
 'psi_ps': 10.571979130943616,
 'psi_sp': 5.535483858110768}

In [52]:
delta()

{'delta_pp': -42.6522302523732,
 'delta_ps': -16.328535512110204,
 'delta_sp': -154.90102034710262}

In [53]:
def Mjr():
    nJr=Jr()
    #Calculus of the reflected Mueller matrix from Jones matrix#
    A=[[1,0,0,1],[-1,0,0,1],[0,1,1,0],[0,1j,-1j,0]]
    Mjr=np.dot(A,np.dot(np.kron(nJr,np.conjugate(nJr)),np.linalg.inv(A)))
    return Mjr

def Mjt():
    nJt=Jt()
    #Calculus of the transmitted Mueller matrix from Jones matrix#
    A=[[1,0,0,1],[-1,0,0,1],[0,1,1,0],[0,1j,-1j,0]]
    Mjt=np.dot(A,np.dot(np.kron(nJt,np.conjugate(nJt)),np.linalg.inv(A)))
    return Mjt

def Mnjr():
    #Calculus of the normalized reflected Mueller matrix 
    nMjr=Mjr()
    Mnjr=nMjr/nMjr[0,0] 
    return Mnjr
    
def Mnjt():
    #Calculus of the normalized transmited Mueller matrix 
    nMjt=Mjt()
    Mnjt=nMjt/nMjt[0,0] 
    return Mnjt

def Tft():
    nJt=Jt()
    ###Calcul of the transmittance including substrate contribution###
    nt=Nsub
    costheta_t=np.sqrt(1-(n_i*np.sin(theta_i)**2/(nt**2))) 
    
    theta_1=np.arccos(costheta_t) 
    theta_2=np.arcsin(Nsub*np.sin(theta_1)/n_i) 

    Rbp=(n_i*np.cos(theta_1)-Nsub*np.cos(theta_2))/(Nsub*np.cos(theta_1)+n_i*np.cos(theta_2)) 
    Rbs=(Nsub*np.cos(theta_1)-n_i*np.cos(theta_2))/(Nsub*np.cos(theta_1)+n_i*np.cos(theta_2)) 
    Tbp=(2*Nsub*np.cos(theta_1))/(Nsub*np.cos(theta_2)+n_i*np.cos(theta_1)) 
    Tbs=(2*Nsub*np.cos(theta_1))/(Nsub*np.cos(theta_1)+n_i*np.cos(theta_2)) 

    Jf=np.matrix([[Tbp,0],[0,Tbs]]) 
    k0=omega/c
    eps=np.matrix([[Nsub**2,0,0],[0, Nsub**2,0],[0, 0,Nsub**2]]) 
    Kxx=n_i*np.sin(theta_i) 
    q=np.sqrt(eps[0,0]-Kxx**2) 
    Jtsub=np.matrix([[np.exp(1j*k0*d2*q),0],[0,np.exp(1j*k0*d2*q)]]) 
    Tt=np.dot(Jf,np.dot(Jtsub,nJt)) 
    Tft=Tt[0,0]*np.conjugate(Tt[0,0]) 
    return Tft